## 텍스트 분류 8529data+twitter+remove(userdic=stopwords+'missprint')

In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from ckonlpy.tag import Twitter #konlpy twitter는 add_dictionary가 없어서, ckonlpy(customized konlpy) 사용.
twitter = Twitter()

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


# ⑴ 자료 읽어오기

In [3]:
df = pd.read_csv('new_data_8529.csv', sep='\t', encoding='UTF-8')

In [4]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [5]:
df.head(3)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   
2               W  2018     20181006         ZZ5         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   
2         KV11          YP     A9          TT        83912   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   
2  [C] 1.현상:Rr RH 커튼 후크  미장착 출고됨.2.조치내용:Rr RH 커튼 ...     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---  
2  X---

In [6]:
df.shape

(8529, 17)

In [3]:
userdic = pd.read_csv('usedic.csv', sep='\t', encoding='UTF-8')

In [4]:
userdic['User Dic.'][0:1]

0    가니쉬
Name: User Dic., dtype: object

In [5]:
userdic = userdic.rename(columns = {'User Dic.':'standard'})
userdic = userdic.rename(columns = {'동의어':'missprint'})
userdic = userdic.rename(columns = {'불용단어':'stops'})

In [6]:
userdic_list = userdic.rename(columns = {'User Dic.':'standard'})
userdic_list = userdic.rename(columns = {'동의어':'missprint'})
userdic_list = userdic.rename(columns = {'불용단어':'stops'})

In [7]:
userdic_list = userdic_list.fillna('')

In [8]:
userdic = userdic.fillna('')

In [9]:
userdic_list[:5]

stops standard missprint
0    결과      가니쉬       가니시
1    공임       가스        개스
2    광주       가죽          
3    넥쏘       간격        간극
4    니로       간격         갭

In [10]:
userdic[:5]

stops standard missprint
0    결과      가니쉬       가니시
1    공임       가스        개스
2    광주       가죽          
3    넥쏘       간격        간극
4    니로       간격         갭

In [11]:
userdic.shape

(656, 3)

In [12]:
userdic_list.shape

(656, 3)

In [13]:
df.columns

Index(['std_clm_type_cd', 'ro_y', 'clm_jgmt_dt', 'ceq_caus_cd', 'ceq_psit_cd',
       'ceq_psit_nm', 'trim_plnt_cd', 'prj_vehl_cd', 'mdl_cd', 'part_sys_cd',
       'repn_part_cd', 'clm_prtl_sbc', '대상 선정', '구분', '책임구', '원인', '검토결과'],
      dtype='object')

In [14]:
df.head(2)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   

                                        clm_prtl_sbc 대상 선정   구분  책임구   원인  \
0  [C] 1.현상:엔진룸 다량의 연료 누유 발생.2.점검:엔진룸 다량의 연료 누유 점...     X  NaN  NaN  NaN   
1                       [C] 1.현상: 리어 도어 앤티 취핑 필림 미장착     X  NaN  NaN  NaN   

   검토결과  
0  X---  
1  X---

In [15]:
df.shape

(8529, 17)

# ⑵ 자료분류현황

In [7]:
category_count = df['대상 선정'].value_counts()
category_count

X    5517
O    3012
Name: 대상 선정, dtype: int64

In [8]:
category_count = df['구분'].value_counts()
category_count

기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [9]:
category_count = df['책임구'].value_counts()
category_count

구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [10]:
category_count = df['원인'].value_counts()
category_count

부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

# ⑶ null값 확인 및 처리하기

In [12]:
# 제거
# df = df[df['대상 선정'].notnull()].copy()
# 다른값으로 채우기
df = df.fillna('N')

In [13]:
df.shape

(8529, 17)

In [16]:
category_count = df['대상 선정'].value_counts()
category_count

X    5517
O    3012
Name: 대상 선정, dtype: int64

In [18]:
category_count = df['구분'].value_counts()
category_count

N          5517
기능(커넥터)    1601
실내부품이음     1050
하체이음        361
Name: 구분, dtype: int64

In [19]:
category_count = df['책임구'].value_counts()
category_count

N       5517
구매      2274
생산공장     724
연구소       14
Name: 책임구, dtype: int64

In [20]:
category_count = df['원인'].value_counts()
category_count

N        5517
부품       2274
조립불량      230
결선불량      225
체결불량      103
기타        100
미결선        37
이종/간섭      21
설계/구조      14
이물          8
Name: 원인, dtype: int64

In [21]:
df[df['구분'].isnull()]

Empty DataFrame
Columns: [std_clm_type_cd, ro_y, clm_jgmt_dt, ceq_caus_cd, ceq_psit_cd, ceq_psit_nm, trim_plnt_cd, prj_vehl_cd, mdl_cd, part_sys_cd, repn_part_cd, clm_prtl_sbc, 대상 선정, 구분, 책임구, 원인, 검토결과]
Index: []

In [22]:
df[df['원인'].isnull()]

Empty DataFrame
Columns: [std_clm_type_cd, ro_y, clm_jgmt_dt, ceq_caus_cd, ceq_psit_cd, ceq_psit_nm, trim_plnt_cd, prj_vehl_cd, mdl_cd, part_sys_cd, repn_part_cd, clm_prtl_sbc, 대상 선정, 구분, 책임구, 원인, 검토결과]
Index: []

In [23]:
df[df['책임구'].isnull()]

Empty DataFrame
Columns: [std_clm_type_cd, ro_y, clm_jgmt_dt, ceq_caus_cd, ceq_psit_cd, ceq_psit_nm, trim_plnt_cd, prj_vehl_cd, mdl_cd, part_sys_cd, repn_part_cd, clm_prtl_sbc, 대상 선정, 구분, 책임구, 원인, 검토결과]
Index: []

In [24]:
df.shape

(8529, 17)

# ⑷ 단어 전처리

In [25]:
# 예측값과 실제값 비교를 위해 컬럼을 하나 더 생성합니다.
df['target_pred'] = df['대상 선정'].astype(str).copy()

In [26]:
df['target_pred'].head()

0    X
1    X
2    X
3    X
4    X
Name: target_pred, dtype: object

## ◈ 트위터 명사추출 및 불용어 제거 

## 1. 정규단어 사전입력하기

In [27]:
from konlpy.tag import Okt,Twitter
twitter = Twitter()

C:\ProgramData\Anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [28]:
from ckonlpy.tag import Twitter

In [29]:
twitter = Twitter()
twitter.add_dictionary(['가니쉬','가스','가죽','간격','간격 단차','간섭','간헐적','감지 센서','개선품','개조','개폐','견인',
                       '경고등','경보음','고객','고장코드','고정','공기압','과다','구멍','그로멧','그리스','글라스','글라스런',
                       '글로브박스','기어','기준','꺾임','나사산','내부','내장재','냉각수','냉간','너트','네비게이션','노브',
                       '노즐','누기','누수','누유','누출','다이얼','단선','단자','단차','단품','닫힘','답력','대쉬','덕트',
                       '데이라이트','데코레이션','도색','도어','도어락','뒤','드레인','들뜸','떨림','떨어짐','라디에이터',
                       '라벨','라이닝','라이선스','라이선스 플레이트','락','란스','램프','랫치','러기지','런찬넬','레귤레이터',
                       '레그','레그타','레버','레벨','레벨링','레인지','레인프','레일','로워암','로컬','롤러','롤러암','루프',
                       '루프랙','룸램프','리모컨','리모트','리어','리어뷰','리저버','리클라이너','리턴','리테이너','리프터',
                       '리프팅','리플렉터','릴레이','릴리즈','링키지','마모','마운팅','마킹','매니폴드','매트','머드가드',
                       '머플러','멀티펑션','멈춤','멤버','모니터','모듈','모터','몰딩','무거움','무드','미결선','미러','미작동',
                       '미장착','미조립','미체결','바디','박음부','배터리','백워닝','백판넬','밸런스','밸브','버클','버튼',
                       '범퍼','베어링','베젤','벤틸레이션','벨트','벨트라인','변속','볼트','부밍음','부조','불규칙','불량',
                       '브라켓','브레이크','블랭킹','블레이드','블로우','블루','블루투스','블루핸즈','비상등','비포장','사이',
                       '사이드','사이드멤버','사제','삼각대','상승','상승 하강','생산','섀시','선글라스','선루프','선바이져',
                       '선쉐이드','세이프티','세차','센서','센터','소손','소켓','속도','송풍','송풍구','쇼버','쇽업','수동',
                       '쉘프','쉬라우드','쉬프트','스마트','스마트키','스위치','스캔','스커프','스크래치','스키드','스트라이커',
                       '스트러트','스티어링','스티어링휠','스포일러','스피커','슬라이딩','시거잭','시동꺼짐','시뮬레이션','시운전',
                       '시트','시트백','시트벨트','실내등','실러','실런트','쏠림','씰링','아울렛','아웃','아웃사이드',
                       '아웃사이드 미러','아이들','악셀','안개등','안테나','안티','암레스트','앞','액츄에이터','액티브',
                       '어라운드뷰','어셈블리','어스','어시스트','언더','언락','언폴딩','얼라인먼트','없음','에어백','에어벤트',
                       '에어컨','에이밍','엔진','엔진룸','엠블럼','여닫음','연동','연료','열림','열선','오버슬램','오버슬램범퍼',
                       '오버헤드','오장착','오체결','오결선','오토','오토큐','오토크루즈','와셔','와이어링','와이퍼','완체결',
                       '외부','요소수','요철','우','운전석','워셔액','원인품','웨더스트립','윈드실드','유닛','유막','유보',
                       '이격','이모빌라이져','이물질','이상','이음','이종품','익스텐션','인도','인버터','인사이드','인사이드 미러',
                       '인슐레이션','인스트','인젝터','인터쿨러','있음','자동','작동','작동 불량','작동력','작동음','잠금',
                       '장착','장치','재발','재부팅','재시동','재이탈','재장착','재조립','재체결','저항','전압','점검','점등',
                       '점멸','정렬','정션','정위치','정차','정체결','제동','제어','조명','조수석','조인트','조작감','조향각',
                       '좌','주름','주차','주행','진공','찬넬','천장','체결','체인지','체커','체크','쳐짐','충격','충전량',
                       '치핑','카울','카페트','캘리퍼','캠버','캡','커넥터','커버','커버링','커튼','커플러','컨트롤','컬럼',
                       '컴프레셔','컴플리트','컵홀더','케이블','코니시티','코드표','콘센트','콘솔','콘티넨탈','콤비','콤비램프',
                       '쿠션','쿨링팬','쿼터','퀵커넥터','크랙','크러쉬','크러쉬패드','크루즈','클램프','클러스터','클럭스프링',
                       '클리너','클립','타이어','탈거','탈부착','태핑스크류','터미널','턴램프','테스트','테이핑','테일게이트',
                       '텐션','토출','토크','통신','트라이앵글홀더','트랜스버스','트랜지스터','트렁크','트레이','트림','틀어짐',
                       '파워','파이프','파킹','파팅','판넬','패드','패들','패스너','패키지트레이','퍼스널','페달','페인트','펜더',
                       '펴짐','평탄로','포그램프','폴딩','풀리','풀핸들','풍절음','퓨즈','프런트','프레임','프로텍터','플랜지',
                       '플러그','플레이트','플로워','피더','픽싱','핀','필러','필러넥','필름','하강','하네스','하드웨어','하우징',
                       '하이드로릭','하이패스','해제','핸들','헤드라이닝','헤드라이트','헤드램프','헤드레스트','헤드업디스플레이',
                       '현상','협력사','호스','혼','홀더','확인','회로','회수량','후드','휘어짐','휠가드','휨','히터','힌지',
                       '엔진룸'],'Noun')

## 2. 특수문자 제거

In [30]:
def preprocessing(text):
    # 개행문자 제거
    text= str(text)
    text = re.sub('\\\\n', ' ', text)
    # 특수문자 제거
    # 특수문자나 이모티콘 등은 때로는 의미를 갖기도 하지만 여기에서는 제거했습니다.
    #text = re.sub('[?.,;:|\)*~`’!^\-_+<>@\#$%&-=#}※]', '', text)
    # 한글, 영문, 숫자만 남기고 모두 제거하도록 합니다.
    #text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    # 한글, 영문만 남기고 모두 제거하도록 합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text)
    return text

In [31]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(preprocessing)

Wall time: 112 ms


## 3. 불용어 제거

In [32]:
# 불용어 제거
def remove_stopwords(text):
    tokens = text.split(' ')
    stops = ['현상', '[C]','현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','(사진첨부)','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F/LH','F/RH','L/H','R/H','L/H리어',
             'R/H리어','LH/RH','LH','RH','(RH)','(LH)','R/LH','R/RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C-10000','C-100','(좌)','(우)','A/S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']
    meaningful_words = [w for w in tokens if not w in stops]
    return ' '.join(meaningful_words) #stops에 안 걸리는 나머지 것들 이어붙이기

In [33]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(remove_stopwords)

Wall time: 486 ms


## 4.명사 추출 → list형태로 추출됨

In [34]:
# 명사추출 
# twitter.pos()함수에다가 dataframe을 직접 대입하면 함수내 내장된 split함수에서 에러발생함 → 그래서 apply를 이용해야 함   
twitter = Twitter() 
def posextract(text):
    text = twitter.nouns(text)
    return text

In [35]:
%time df['clm_prtl_sbc'] = df['clm_prtl_sbc'].apply(posextract)

Wall time: 26.9 s


In [36]:
df['clm_prtl_sbc'].head()

0    [엔진, 룸, 다량, 연료, 누유, 발생, 점검, 엔진, 룸, 다량, 연료, 누유,...
1                           [리, 도어, 앤티, 취핑, 필림, 미장, 착]
2              [커튼, 후크, 미장, 착, 출고, 커튼, 후크, 장착후, 출고, 함]
3             [차량, 출고, 시, 웨즈, 스트립, 고무, 주름, 발생, 교환, 조치]
4                      [몰딩, 내부, 고무, 미조, 립, 상태, 몰딩, 조립]
Name: clm_prtl_sbc, dtype: object

In [37]:
df.head(2)

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd    ceq_psit_nm  \
0               W  2017     20170811         ZZ7         B11  조립 문제_미장착/오장착   
1               W  2018     20180822         ZZ7         B11  조립 문제_미장착/오장착   

  trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
0         KV11          CK     J5          TT        82305   
1         KV11          YP     A9          TT        84211   

                                        clm_prtl_sbc 대상 선정 구분 책임구 원인  검토결과  \
0  [엔진, 룸, 다량, 연료, 누유, 발생, 점검, 엔진, 룸, 다량, 연료, 누유,...     X  N   N  N  X---   
1                         [리, 도어, 앤티, 취핑, 필림, 미장, 착]     X  N   N  N  X---   

  target_pred  
0           X  
1           X

## 5. Typo 수정

In [38]:
df['clm_prtl_sbc'].shape

(8529,)

In [39]:
df = df.reindex()

In [41]:
data = df['clm_prtl_sbc']

### ① list ▶ string으로 변환

In [42]:
for i in range(0,8529):
    str_data = ",".join(data[i])
    data.iloc[i] = str_data  # string 변환

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [43]:
userdic = userdic.astype(str).copy()

In [44]:
type(userdic)

pandas.core.frame.DataFrame

In [45]:
type(userdic.loc[1,'missprint'])

str

In [46]:
data[1]

'리,도어,앤티,취핑,필림,미장,착'

In [47]:
type(data[1])

str

In [48]:
data[0]

'엔진,룸,다량,연료,누유,발생,점검,엔진,룸,다량,연료,누유,점검,엔진,룸,프론트,저압,라인,연료,공급,파이프,중,고압,펌프,연결,저압,라인,퀵,컨넥터,오,체결,랭킹,중,다량,료,엔진,룸,내부,고압,연료,펌프,하단,바닥,누유,현상,발생,함,고압,펌프,저압,라인,퀵,컨넥터,오,체결,연료,펌프,구동,중,이탈,다량,연료,엔진,룸및,차량,다닥,누유,사항,임,고압,펌측,퀵,턴텍터,체결,및,엔진,룸,가솔린,오염,부위,세척,시동,후,누유,확인,테스트,요망,사항,해당,저압,파이프,이탈,시,파이프,하단,부위,익,죠스,트,매니폴드,이,어서,가솔린,연료,발화,인하,회재,발생,수,중요,부위,조립,철저,관리,필요,사항,임'

### ② Typo 카운트 체크해 보기

In [49]:
data[0].count('엔진')

6

In [55]:
i=0
j=0
df_cnt2 = pd.DataFrame(columns=['word_cnt_all'])
for j in range(0,656):
    for i in range(0,8529):
        my_cnt = data[i].count(userdic_list.loc[j,'missprint'])
        sub_data = {'word_cnt':[my_cnt]}
        df_cnt = pd.DataFrame(sub_data)
        df_sub_cnt = df_cnt.sum()
        i+=1
    df_cnt2['word_cnt_all'] = df_sub_cnt
    j+=1

In [56]:
df_cnt2.head()

word_cnt_all
word_cnt            38

In [83]:
df_cnt2 = pd.DataFrame(columns=['word_cnt_all'])

my_cnt = data[0].count(userdic_list.loc[3,'missprint'])
sub_data = [{'word_cnt':my_cnt}]
df_cnt. = pd.DataFrame(sub_data)
df_cnt

df = pd.DataFrame([[1, 2], [4, 5], [7, 8]],index=['cobra', 'viper', 'sidewinder'], columns=['max_speed', 'shield'])


word_cnt
0         0

### ③ Typo ▶ standard 단어로 변경하기

In [ ]:
# 참고코너 Trials and Errors

### 실패한 코드1
# 단어 오자 수정
# index1=0
# index2=0
# for index1 in range(0,656):
#     for index2 in range(0,8529):
#         if (userdic.loc[index1,'missprint'] == data[index2]):
#             data.loc[index2] = data[index2].replace(userdic.loc[index1,'missprint'],userdic.loc[index1,'standard'])
#         elif data[index2].count(userdic.loc[index1,'missprint']) == 0 :
#             count = data[index2].count(userdic.loc[index1,'missprint'])
#             data.loc[index2] = data[index2].replace(userdic.loc[index1,'missprint'],userdic.loc[index1,'standard'],count)
#         else :
#              index2+=1
#     index1+=1


##### 실패한 코드2
# 단어 오자 수정
# 시간이 오래 걸려서 userdic내 특정 단어만 오자 수정이 가능한지 시험
# idx=0
# for idx in range(0,8529):
#     miss1 = userdic.loc[3,'missprint']
#     stan1 = userdic.loc[3,'standard']
#     if (miss1 == data[idx]):
#         data.loc[idx] = data[idx].replace(miss1,stan1)
#     elif data[idx].count(miss1) == 0 :
#         count = data[idx].count(miss1)
#         data.loc[idx] = data[idx].replace(miss1,stan1,count)
#     else :
#         idx+=1


#### 실패한 코드3
# 각 행에 오자(typo=missprint)가 몇 개 있는 확인
# idx = 0
# udx = 0
# for udx in range(0,656):
#     missprint = userdic_list.loc[udx,'missprint']
#     for idx in range(0,8529):
#         data[idx] == missprint:
#         count = data[idx].count(missprint)
#         total_cnt += count
#         else:
#             idx += 1
#     print(udx,count)
# udx =+ 1


### 목적한 바대로 실행되지 않는 코드
# idx = 0
# for idx in range(0,8529):
#     count = data[idx].count(userdic_list.loc[4,'missprint'])
#     print(count)


### 참고 코드
# 만들어 놓은 단어 사전 데이터프레임 연관단어를 카운트 하는 코드
# data[0].count(userdic_list.loc[3,'missprint'])
# 문자열내 특정단어 카운트하기 코드
# data[0].count('엔진')


# 누적합 구하는 코드
# def my_sum(cnt):    
#     total = 0  # 합계
#     for n in cnt:
#         total += n
#     return total

# ⑹ 학습데이터와 테스트데이터 만들기

In [289]:
df = df.reindex() #index 재설정. reindex:새로운 색인에 맞도록 객체를 새로 생성하는 기능.

In [290]:
df.shape

(8529, 18)

In [291]:
from sklearn.model_selection import train_test_split

In [292]:
df_train_v,df_test_v = train_test_split(df, test_size = 0.1, random_state=123)

In [293]:
df_train = df_train_v.copy()
df_test = df_test_v.copy()

In [294]:
df_train.shape

(7676, 18)

In [295]:
df_test.shape

(853, 18)

# ⑺ 단어 벡터화

In [296]:
stop_dict = ['현상', '현상및점검','현    상','현 상','점검점검','점검내용','조치내용','점검점검내용','현상및조치','C','원인',
             '점검/원인','사진첨부','사진참조','점검및원인','동영상참조','동영상','대리점','고객','고객에게','상기',
             '차량','상기차량은','자동차','당차량은','당차량','당','차량은','점검및조치내용','F LH','F RH','L H','R H','L H리어',
             'R H리어','LH RH','LH','RH','RH','LH','R LH','R RH','리어RH','리어LH','LH쪽','RH쪽','Rr','RR','FRT','Frt',
             'frt','rr','C 10000','C 100','좌','우','A S','결과','공임','광주','넥쏘','니로','레이','렌트','리무진',
             '모닝','모하비','벨로스터','사진','상기','서비스','소하','스타렉스','스팅어','스포티지','신차','싼타페','써비스',
             '쏘나타','쏘렌토','쏘울','아반떼','아산','아이오닉','언덕','엑센트','열','울산','임시 넘버','임시 번호','입고',
             '저머검내용','점검결과','조치결과','주차장','증상','지하','철도','첨부','첨부파일','청구','출고','카 마스터','카니발',
             '카렌스','코나','클레임','투싼','팰리세이드','하이리무진','하이브리드','현     상','현    상','현   상','현  상',
             '현 상','현산','화성','그랜저','새벽','새차','야간','주간','그랜져','주재원','아침','밤, 심야','낮,낫','주제원']

In [297]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer = 'word', # 캐릭터 단위로 벡터화 할 수도 있습니다.
                             tokenizer = None, # 토크나이저를 따로 지정해 줄 수도 있습니다.
                             preprocessor = None, # 전처리 도구
                             stop_words = stop_dict, # 불용어 nltk등의 도구를 사용할 수도 있습니다.
                             min_df = 1, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                             ngram_range=(1, 3), # BOW의 단위를 1~3개로 지정합니다.
                             max_features = 20000 # 만들 피처의 수, 단어의 수가 된다.
                            )
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 3), preprocessor=None,
        stop_words=['현상', '현상및점검', '현    상', '현 상', '점검점검', '점검내용', '조치내용', '점검점검내용', '현상및조치', 'C', '원인', '점검/원인', '사진첨부', '사진참조', '점검및원인', '동영상참조', '동영상', '대리점', '고객', '고객에게', '상기', '차량', '상기차량은', '자동차', '당차량은', '당차량', '당', '차량은', '점검및조치내용', 'F LH', 'F RH', 'L H', 'R H', 'L H리어', 'R H리어', 'LH RH', 'LH', 'R...'현  상', '현 상', '현산', '화성', '그랜저', '새벽', '새차', '야간', '주간', '그랜져', '주재원', '아침', '밤, 심야', '낮,낫', '주제원'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [298]:
#%%time 
train_feature_vector = vectorizer.fit_transform(df_train['clm_prtl_sbc'])
train_feature_vector.shape

(7676, 20000)

In [299]:
#%%time 
test_feature_vector = vectorizer.fit_transform(df_test['clm_prtl_sbc'])
test_feature_vector.shape

(853, 20000)

In [300]:
type(train_feature_vector)

scipy.sparse.csr.csr_matrix

In [301]:
vocab = vectorizer.get_feature_names()
print(len(vocab))
vocab[:10]

20000


['가감',
 '가감 실내',
 '가감 실내 대쉬보드',
 '가감 저속',
 '가감 저속 등판',
 '가끔',
 '가끔 소음',
 '가끔 소음 허브',
 '가끔 재현',
 '가끔 재현 발생']

In [302]:
len(vocab)

20000

In [304]:
dist = np.sum(train_feature_vector, axis=0)
dist.shape

(1, 20000)

In [305]:
dist_pd = pd.DataFrame(dist, columns=vocab)
dist_pd

가감  가감 실내  가감 실내 대쉬보드  가감 저속  가감 저속 등판  가끔  가끔 소음  가끔 소음 허브  가끔 재현  \
0   4     12           3      5         4  33      5         8      4   

   가끔 재현 발생    ...     힌지 뒤쪽  힌지 뒤쪽 스톱  힌지 부위  힌지 부위 조수  힌지 스톱  힌지 스톱 볼트  \
0       188    ...         3         9      5         5      6        36   

   힌지 스트라이크  힌지 스트라이크 조정  힌지 조정  힌지 조정 작업  
0         3            3      3         5  

[1 rows x 20000 columns]

In [306]:
dist_pd.columns

Index(['가감', '가감 실내', '가감 실내 대쉬보드', '가감 저속', '가감 저속 등판', '가끔', '가끔 소음',
       '가끔 소음 허브', '가끔 재현', '가끔 재현 발생',
       ...
       '힌지 뒤쪽', '힌지 뒤쪽 스톱', '힌지 부위', '힌지 부위 조수', '힌지 스톱', '힌지 스톱 볼트',
       '힌지 스트라이크', '힌지 스트라이크 조정', '힌지 조정', '힌지 조정 작업'],
      dtype='object', length=20000)

In [307]:
dist_pd_t = dist_pd.T
dist_pd_t.head()

0
가감           4
가감 실내       12
가감 실내 대쉬보드   3
가감 저속        5
가감 저속 등판     4

In [308]:
dist_pd_t.index.name = "word"

In [309]:
dist_pd_t[:2]

0
word     
가감      4
가감 실내  12

In [310]:
dist_pd_t.reset_index(level='word', inplace=True)

In [311]:
dist_pd_t[:3]

word   0
0          가감   4
1       가감 실내  12
2  가감 실내 대쉬보드   3

In [312]:
dist_pd_t.rename(columns={0:'count'}, inplace = True)

In [313]:
dist_pd_t[:200]

word  count
0            가감      4
1         가감 실내     12
2    가감 실내 대쉬보드      3
3         가감 저속      5
4      가감 저속 등판      4
5            가끔     33
6         가끔 소음      5
7      가끔 소음 허브      8
8         가끔 재현      4
9      가끔 재현 발생    188
10           가나      4
11        가나 왼쪽      3
12     가나 왼쪽 소리     37
13       가나 음으로      3
14    가나 음으로 스티      6
15           가능      4
16        가능 거리      3
17     가능 거리 달라      4
18     가능 거리 변동      6
19          가니쉬      4
20       가니쉬 고정      3
21    가니쉬 고정 불량      3
22       가니쉬 고함      3
23    가니쉬 고함 시부      3
24       가니쉬 교체      9
25       가니쉬 교환      7
26    가니쉬 교환 정상      4
27    가니쉬 교환 조치      5
28      가니쉬 글래스      8
29   가니쉬 글래스 교환      3
..          ...    ...
170       가지 고장      3
171    가지 고장 점등     38
172          각종      6
173       각종 경고      4
174    각종 경고 점등     23
175       각종 고등      3
176    각종 고등 점등      3
177       각종 소음      5
178    각종 소음 부분      8
179          간격      4
180       간격 소로     50
181    간격 소로 소음      3
182       간격 수정      3
183   간격 수정 시운전      9
184       간격 협소      6
185    간격 협소 불만      3
186          간극      4
187       간극 도어     10
188    간극 도어 휀더      6
189       간극 불량      3
190    간극 불량 소음     23
191    간극 불량 점검      3
192    간극 불량 조정     13
193     간극 슬라이딩      3
194  간극 슬라이딩 도어     47
195       간극 아래      4
196    간극 아래 범퍼      3
197       간극 음원     15
198    간극 음원 범퍼      3
199       간극 조정      5

[200 rows x 2 columns]

In [260]:
dist_pd_t = dist_pd_t.replace(to_replace="간극",value="간격")
df1 = df.replace(to_replace="개스",value="가스")
df1 = df.replace(to_replace="간극",value="간격")
df1 = df.replace(to_replace="간단차",value="간격 단차")
df1 = df.replace(to_replace="갭단차",value="간격 단차")
df1 = df.replace(to_replace="단간차",value="간격 단차")
df1 = df.replace(to_replace="간혈적",value="간헐적")
df1 = df.replace(to_replace="감지센서",value="감지 센서")
df1 = df.replace(to_replace="울트라센서",value="감지 센서")
df1 = df.replace(to_replace="울트라 센서",value="감지 센서")
df1 = df.replace(to_replace="개페",value="개폐")
df1 = df.replace(to_replace="개패",value="개폐")
df1 = df.replace(to_replace="문구",value="고장코드")
df1 = df.replace(to_replace="맨트",value="고장코드")
df1 = df.replace(to_replace="멘트",value="고장코드")

# tf-idf 가중치 적용

In [314]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
transformer

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [315]:
#%%time 
train_feature_tfidf = transformer.fit_transform(train_feature_vector)
train_feature_tfidf.shape

(7676, 20000)

In [316]:
#%%time 
test_feature_tfidf = transformer.fit_transform(test_feature_vector)
test_feature_tfidf.shape

(853, 20000)

# 머신러닝 : 학습

In [317]:
from sklearn.ensemble import RandomForestClassifier

# 랜덤포레스트 분류기를 사용
forest = RandomForestClassifier(
    n_estimators = 100, n_jobs = -1, random_state=2018)
forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=False, random_state=2018, verbose=0,
            warm_start=False)

In [318]:
# 학습에 사용할 y_label 을 넣어줍니다.
# 어떤 분야의 청원인지 예측할 것이기 때문에 category를 넣어줍니다.
# y_label2 = df_train['caus_pred']
y_label = df_train['target_pred']
# y_label3 = df_train['charge_pred']
%time forest = forest.fit(train_feature_tfidf, y_label)
# %time forest2 = forest.fit(train_feature_tfidf, y_label2)
# %time forest3 = forest.fit(train_feature_tfidf, y_label3)

Wall time: 1.64 s


# 모델평가

In [319]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
k_fold = KFold(n_splits=5, shuffle=True, random_state=0)

scoring = 'accuracy'
%time score = cross_val_score(forest, train_feature_vector, y_label, cv=k_fold, n_jobs=-1, scoring=scoring)
score

Wall time: 31.8 s


array([0.79492188, 0.80260586, 0.80065147, 0.81758958, 0.8       ])

In [320]:
round(np.mean(score)*100,2)

80.32

# 예측

In [321]:
# 테스트 데이터를 넣고 예측합니다.
y_pred = forest.predict(test_feature_vector)
y_pred[:3]

array(['X', 'X', 'X'], dtype=object)

In [322]:
y_pred.shape

(853,)

In [323]:
# 예측 결과를 저장하기 위해 데이터프레임에 담아 줍니다.
output = pd.DataFrame(data={'category_pred':y_pred})
output.head(10)

category_pred
0             X
1             X
2             X
3             X
4             X
5             X
6             X
7             X
8             X
9             X

In [324]:
df_test['category_pred'] = y_pred

# 결과검증

In [325]:
df_test['pred_diff'] = 0
df_test['pred_diff'] = (df_test['대상 선정'] == df_test['category_pred'] ) == 1
df_test['pred_diff'] = df_test['pred_diff'].astype(int)
df_test.head()

std_clm_type_cd  ro_y  clm_jgmt_dt ceq_caus_cd ceq_psit_cd   ceq_psit_nm  \
8400               W  2015     20151214         ZZ3         V91  차체 진동/떨림_주행중   
2485               W  2015     20150629         ZZ4         I11     작동불량_작동안됨   
6990               W  2017     20170329         ZZ5         Q55     소음/이음_주행중   
5899               W  2017     20171002         ZZ3         Q52    소음/이음_공회전시   
2186               W  2015     20150318         ZZ4         I11     작동불량_작동안됨   

     trim_plnt_cd prj_vehl_cd mdl_cd part_sys_cd repn_part_cd  \
8400         KV11          KH     DH          CC        52910   
2485         KV11          YP     A9          AC        96985   
6990         KV11          YP     A9          CC        58250   
5899         KV11          YP     A9          TT        89600   
2186         KV11          KH     DH          TT        92800   

                                           clm_prtl_sbc 대상 선정       구분 책임구  \
8400  주행,중,핸들,타이어,바란,스,점검,바란,스,점검,시,그,바,란스,바란,스,점검,조...     X        N   N   
2485  클러스터,온도,표시,불량,점검,하니,배선,커넥터,정상,학인,온도,센서,품질,불량,확...     O  기능(커넥터)  구매   
6990  주행,중,소,음으로,및,주행,중,뜨드득,소음,발생,및,후진,시,브레이크,현상,점검,...     O     하체이음  구매   
5899            시트,좌,이동,시,체결,구리,스,및,교정,작업,동일,현상,지속,교환,함     X        N   N   
2186                       운전,쪽,무드등,점등,불량,오버헤드,콘솔,램프,교환     O  기능(커넥터)  구매   

      원인             검토결과 target_pred category_pred  pred_diff  
8400   N             X---           X             X          1  
2485  부품  O-기능(커넥터)-구매-부품           O             X          0  
6990  부품     O-하체이음-구매-부품           O             X          0  
5899   N             X---           X             X          1  
2186  부품  O-기능(커넥터)-구매-부품           O             X          0

In [326]:
diff_sum = df_test['pred_diff'].sum()
diff_sum

532

In [327]:
y_pred_len = len(y_pred)
y_pred_len

853

# 정확도

In [328]:
Acy = round(diff_sum/y_pred_len*100,1)
Acy

62.4